# Pytowr 与 vrep 接口文档
这里的基本思路是, 用pytowr跑出的结果, 指导vrep的结果. pytowr获得的是特定的时间点,机器人的身体和脚相对地面坐标系的坐标, vrep里面一般设置的是: 机器人的脚相对于机器人身体的坐标. 所以理论上我们进行坐标系转换再赋过去即可.

但是由于种种问题, 目前我测试中让机器人上楼梯仍然会爆炸,没有成功过.

首先遇到太高的台阶迈腿可能卡主,可能原因有:
- 足尖点的定义不一样, 机器人算的是足尖点,但是我没有考虑机器人脚下面那个球的大小
- 直线变化导致本来可以绕上去的曲线被我变成spline, 选取的点的连线与地面相交
- towr本身没有考虑到地形的边缘
解决的方法基本也就是timescale设得小一些, 抬腿的时候考虑一下脚下面的球的半径, 实在不行地图设得比搜的低一点

其次机器人vrep中平地走的到位置与搜到的目标位置不符,可能的原因是:\
在机器人抬腿的时候,在towr中认为机器人的腿抬起来了与地面没有作用了,但是在vrep中机器人腿虽然抬了,但是和地面仍有作用力(因为并不是纯硬的),所以向前伸抬起的腿的时候仍略微向后推机器人,导致误差.

还有机器人无理由爆炸的问题,可能是使力的角度不对? 我发现有时候机器人的腿比斜的时候,仍向地面用力,爆炸了,(相当于人的脚内侧着地然后夹腿)

>这里面使用的pkl,但是只是为了与vrep调参方便,实际上完全不必

In [1]:
from powerGait import *

Program started
Connected to remote API server


## Restart virtual environmment

In [8]:
vrep.simxStopSimulation(clientID, vrep.simx_opmode_blocking)
time.sleep(5)
status = vrep.simxStartSimulation(clientID, vrep.simx_opmode_blocking)
# recover()

### Load the pkl dump

In [3]:
import pickle as pkl
# with open("pytowrDump.pkl","rb") as f:            # walk straight to 0.5,0
# with open("pytowrDump_turnAround.pkl","rb") as f: # walk to 0.5,0 and turn 90 degree
# with open("pytowrDumpUpstair.pkl","rb") as f:     # try to climb a 0.5 stair at 1.0
# with open("pytowrDumpstair0.3.pkl","rb") as f:    # try to climb a 0.3 stair at 1.0
with open("pytowrDump_S30_T001.pkl","rb") as f:     # try to climb a 0.3 stair at 1.0 the time scale is 0.01
# with open("pytowrDump_Cparallel_S03_T001.pkl","rb") as f: # try to climb a 0.3 stair at 1.0 the time scale is 0.01 with elongation constraints
    pos = pkl.load(f)
print(pos[0])

(0.0, (0.0, 0.0, 0.45), (0.0, 0.0, 0.0), (((0.528, 0.304, 0.0), 1), ((0.0, 0.609, 0.0), 1), ((-0.528, 0.304, 0.0), 1), ((0.528, -0.304, 0.0), 1), ((0.0, -0.609, 0.0), 1), ((-0.528, -0.304, 0.0), 1)))


In [45]:
# check the pos 0 with the current robot posture
print("body lin",vrep.simxGetObjectPosition(clientID, BCS, -1, vrep.simx_opmode_oneshot_wait)[1])
print("body ang",vrep.simxGetObjectOrientation(clientID, BCS, -1, vrep.simx_opmode_oneshot_wait)[1])
for i in range(6):
    print("foot pos%d"%i,vrep.simxGetObjectPosition(clientID, S1[i], -1, vrep.simx_opmode_oneshot_wait)[1])

body lin [0.9352151155471802, -0.06795454770326614, 0.3022192418575287]
body ang [-0.0004745058831758797, 0.00030683827935718, 1.4615472555160522]
foot pos0 [0.7286672592163086, 0.4912768602371216, 0.027189865708351135]
foot pos1 [0.3379303216934204, 0.02602500468492508, 0.02756670117378235]
foot pos2 [0.6170831918716431, -0.5689376592636108, 0.02777153253555298]
foot pos3 [1.3444076776504517, 0.3952353298664093, 0.026982754468917847]
foot pos4 [1.5904252529144287, -0.1395203173160553, 0.027124375104904175]
foot pos5 [1.1430811882019043, -0.638715386390686, 0.027585431933403015]


### The turn function
transform the position in global axis to robot axis

In [5]:
from scipy.spatial.transform import Rotation as Rot
def turnDeg(rot_vec, m):
    r = Rot.from_rotvec(rot_vec)
    return r.apply(m)
turnDeg(np.array([0,0,1.57]),np.array([[1,0,1.57],[0,1,-2]]))

array([[ 7.96326711e-04,  9.99999683e-01,  1.57000000e+00],
       [-9.99999683e-01,  7.96326711e-04, -2.00000000e+00]])

In [9]:
for i,p in enumerate(pos):
    print(i)
    body_lin = np.array(p[1])
    footpos = np.array([f[0]-body_lin for f in p[3] ] )
    footpos = turnDeg(-np.array(p[2]),footpos)
    footpos+=np.array([0,0,0.02]) # The radius of the ped ball 也就是transto的坐标是设置机器人那个足尖点的坐标,但是足尖点下面与地面接触的是一个球,需要把这个球的大小考虑进来
    # footpos.shape
    transTo(footpos)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


KeyboardInterrupt: 